## Bayesian AGN Decomposition Analysis for SDSS Spectra (BADASS)
### Example: MANGA IFU Cube

This notebook shows how you can fit MANGA IFU data using BADASS and multiprocessing.

####  Remington O. Sexton$^{1}$, Sara M. Doan$^{2}$, William Matzko$^{2}$ Michael A. Reefe$^{2}$, 
$^{1}$United States Naval Observatory, $^{2}$George Mason University


In [1]:
import glob
import time
import natsort
from IPython.display import clear_output
import os
import sys
import psutil
import shutil
import pathlib
import natsort
# Import BADASS here
BADASS_DIR = pathlib.Path(os.getcwd()).resolve().parent
sys.path.insert(1,str(BADASS_DIR))
import badass as badass
import badass_check_input
import badass_ifu as ifu

from IPython.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))

Download MANGA cube if it doesn't exist (it might take awhile since its ~512 Mb!)

In [2]:
# Set to 1 to download this data (you only need to do this once!)
if 0: 
    if not os.path.exists(BADASS_DIR.joinpath("example_spectra","MANGA","manga-9485-12705-LOGCUBE.fits")):
        import requests
        import gzip
        # manga-9485-12705-LOGCUBE.fits
        plate     = 9485
        ifudesign = 12705
        db_url        = 'https://data.sdss.org/sas/dr17/manga/spectro/redux/v3_1_1/%d/stack/' % (plate)
        cube_filename = 'manga-%d-%d-LOGCUBE.fits.gz' % (plate,ifudesign)
        
        r = requests.get(db_url+cube_filename, verify=False)
    
        with open(str(BADASS_DIR.joinpath("example_spectra","MANGA","%s" % cube_filename)), 'wb') as f:
            f.write(r.content)
    
        # Retrieve HTTP meta-data
        print(r.status_code)
        # Unzip the cube
        with gzip.open(str(BADASS_DIR.joinpath("example_spectra","MANGA","%s" % cube_filename)), 'rb') as f_in:
                with open(str(BADASS_DIR.joinpath("example_spectra","MANGA","%s" % cube_filename[:-3])), 'wb') as f_out:
                    shutil.copyfileobj(f_in, f_out)
    

### BADASS Options

In [3]:
################################## Fit Options #################################
# Fitting Parameters
fit_options={
"fit_reg"    : (4400,5500),# Fitting region; Note: Indo-US Library=(3460,9464)
"good_thresh": 0.0, # percentage of "good" pixels required in fig_reg for fit.
"mask_bad_pix": False, # mask pixels SDSS flagged as 'bad' (careful!)
"mask_emline" : False, # automatically mask lines for continuum fitting.
"mask_metal": False, # interpolate over metal absorption lines for high-z spectra
"fit_stat": "RCHI2", # fit statistic; ML = Max. Like. , OLS = Ordinary Least Squares, RCHI2 = reduced chi2
"n_basinhop": 10, # Number of consecutive basinhopping thresholds before solution achieved
"test_lines": False, # Perform line/configuration testing for multiple components
"max_like_niter": 10, # number of maximum likelihood iterations
"output_pars": False, # only output free parameters of fit and stop code (diagnostic)
"cosmology": {"H0":70.0, "Om0": 0.30}, # Flat Lam-CDM Cosmology
}
################################################################################

########################### MCMC algorithm parameters ##########################
mcmc_options={
"mcmc_fit"    : False, # Perform robust fitting using emcee
"nwalkers"    : 100,  # Number of emcee walkers; min = 2 x N_parameters
"auto_stop"   : False, # Automatic stop using autocorrelation analysis
"conv_type"   : "all", # "median", "mean", "all", or (tuple) of parameters
"min_samp"    : 1000,  # min number of iterations for sampling post-convergence
"ncor_times"  : 10.0,  # number of autocorrelation times for convergence
"autocorr_tol": 10.0,  # percent tolerance between checking autocorr. times
"write_iter"  : 100,   # write/check autocorrelation times interval
"write_thresh": 100,   # iteration to start writing/checking parameters
"burn_in"     : 1500, # burn-in if max_iter is reached
"min_iter"    : 1000, # min number of iterations before stopping
"max_iter"    : 2500, # max number of MCMC iterations
}
################################################################################

############################ Fit component op dtions #############################
comp_options={
"fit_opt_feii"     : True, # optical FeII
"fit_uv_iron"      : False, # UV Iron 
"fit_balmer"       : False, # Balmer continuum (<4000 A)
"fit_losvd"        : True, # stellar LOSVD
"fit_host"         : False, # host template
"fit_power"        : True, # AGN power-law
"fit_poly"         : True, # Add polynomial continuum component
"fit_narrow"       : True, # narrow lines
"fit_broad"        : False, # broad lines
"fit_absorp"       : False, # absorption lines
"tie_line_disp"    : False, # tie line widths (dispersions)
"tie_line_voff"    : False, # tie line velocity offsets
}

# Line options for each narrow, broad, and absorption.
narrow_options = {
#     "amp_plim": (0,1), # line amplitude parameter limits; default (0,)
    "disp_plim": (0,500), # line dispersion parameter limits; default (0,)
    "voff_plim": (-500,500), # line velocity offset parameter limits; default (0,)
    "line_profile": "gaussian", # line profile shape*
    "n_moments": 4, # number of higher order Gauss-Hermite moments (if line profile is gauss-hermite, laplace, or uniform)
}

broad_options ={
#     "amp_plim": (0,40), # line amplitude parameter limits; default (0,)
    "disp_plim": (500,3000), # line dispersion parameter limits; default (0,)
    "voff_plim": (-1000,1000), # line velocity offset parameter limits; default (0,)
    "line_profile": "voigt", # line profile shape*
    "n_moments": 4, # number of higher order Gauss-Hermite moments (if line profile is gauss-hermite, laplace, or uniform)
}

absorp_options = {
#     "amp_plim": (-1,0), # line amplitude parameter limits; default (0,)
#     "disp_plim": (0,10), # line dispersion parameter limits; default (0,)
#     "voff_plim": (-2500,2500), # line velocity offset parameter limits; default (0,)
    "line_profile": "gaussian", # line profile shape*
    "n_moments": 4, # number of higher order Gauss-Hermite moments (if line profile is gauss-hermite, laplace, or uniform)        
}

# Choices for line profile shape include 'gaussian', 'lorentzian', 'voigt',
# 'gauss-hermite', 'laplace', and 'uniform'
################################################################################

test_options = {
"test_mode":"line",
"lines": [["NA_OIII_5007","NA_OIII_4960","NA_H_BETA"]], # The lines to test
"ranges":[(4400,5500)], # The range over which the test is performed must include the tested line
# "groups": [["NA_OIII_5007","NA_OIII_4960","NA_H_BETA"],["BR_H_BETA"]], # groups of line associated lines including the lines being tested
"metrics": ["BADASS", "ANOVA", "CHI2_RATIO", "AON"],# Fitting metrics to use when determining the best model
"thresholds": [0.95, 0.95, 0.10, 3.0],
"conv_mode": "any", # "any" single threshold satisfies the solution, or "all" must satisfy thresholds
"auto_stop":False, # automatically stop testing once threshold is reached; False test all no matter what
"full_verbose":False, # prints out all test fitting to screen
"plot_tests":True, # plot the fit of each model comparison
"force_best":True, # this forces the more-complex model to have a fit better than the previous.
"continue_fit":True, # continue the fit with the best chosen model
}

########################### Emission Lines & Options ###########################
# If not specified, defaults to SDSS-QSO Emission Lines (http://classic.sdss.org/dr6/algorithms/linestable.html)
################################################################################
# User lines overrides the default line list with a user-input line list!
user_lines = {
    "NA_H_BETA"      :{"center":4862.691,"amp":"free","disp":"NA_OIII_5007_DISP","voff":"free","line_type":"na","label":r"H$\beta$","ncomp":1,},
    "NA_H_BETA_2"    :{"center":4862.691,"amp":"NA_H_BETA_AMP*NA_OIII_5007_2_AMP/NA_OIII_5007_AMP","disp":"NA_OIII_5007_2_DISP","voff":"NA_OIII_5007_2_VOFF","line_type":"na","ncomp":2,"parent":"NA_H_BETA"},

    "NA_OIII_4960"   :{"center":4960.295,"amp":"(NA_OIII_5007_AMP/2.98)","disp":"NA_OIII_5007_DISP","voff":"NA_OIII_5007_VOFF","line_type":"na","label":r"[O III]","ncomp":1,},
    "NA_OIII_4960_2" :{"center":4960.295,"amp":"(NA_OIII_5007_2_AMP/2.98)","disp":"NA_OIII_5007_2_DISP","voff":"NA_OIII_5007_2_VOFF","line_type":"na","ncomp":2,"parent":"NA_OIII_4960"},

    "NA_OIII_5007"   :{"center":5008.240,"amp":"free","disp":"free","voff":"free","line_type":"na","label":r"[O III]","ncomp":1,},
    "NA_OIII_5007_2" :{"center":5008.240,"amp":"free","disp":"free","voff":"free","line_type":"na","ncomp":2,"parent":"NA_OIII_5007"},

    # "BR_H_BETA"      :{"center":4862.691,"amp":"free","disp":"free","voff":"free","line_type":"br","ncomp":1,},
}
user_constraints = [
    # ("NA_OIII_5007_AMP","NA_OIII_5007_2_AMP"),
    # ("NA_OIII_5007_2_DISP","NA_OIII_5007_DISP"),
]
# User defined masked regions (list of tuples)
user_mask = [
#     (4840,5015),
]

# Combined lines; define a composite line and calculate
# its combined parameters.  These are automatically
# generated for lines with multiple components (parent+child lines)
combined_lines = {
    # "H_BETA_COMP"   :["NA_H_BETA","BR_H_BETA"],
}
########################## LOSVD Fitting & Options #############################
# For direct fitting of the stellar kinematics (stellar LOSVD), one can 
# specify a stellar template library (Indo-US or Vazdekis 2010).
# One can also hold velocity or dispersion constant to avoid template
# convolution during the fitting process.
################################################################################

losvd_options = {
"library"   : "IndoUS", # Options: IndoUS, Vazdekis2010
"vel_const" :  {"bool":False, "val":0.0},
"disp_const":  {"bool":False, "val":250.0},
}

########################## SSP Host Galaxy Template & Options ##################
# The default is zero velocity, 100 km/s dispersion 10 Gyr template from 
# the eMILES stellar library. 
################################################################################

host_options = {
"age"       : [1.0,5.0,10.0], # Gyr; [0.09 Gyr - 14 Gyr] 
"vel_const" : {"bool":False, "val":0.0},
"disp_const": {"bool":False, "val":150.0}
}

########################### AGN power-law continuum & Options ##################
# The default is a simple power law.
################################################################################

power_options = {
"type" : "simple" # alternatively, "broken" for smoothly-broken power-law
}

########################### Polynomial Continuum Options #######################
# Options for an additive legendre polynomial or multiplicative polynomial to be 
# included in the fit.  NOTE: these polynomials do not include the zeroth-order
# (constant) term to avoid degeneracies with other continuum components.
################################################################################

poly_options = {
"apoly" : {"bool": True , "order": 3}, # Legendre additive polynomial 
"mpoly" : {"bool": False, "order": 3}, # Legendre multiplicative polynomial 
}

############################### Optical FeII options ###############################
# Below are options for fitting optical FeII.  For most objects, you don't need to 
# perform detailed fitting on FeII (only fit for amplitudes) use the 
# Veron-Cetty 2004 template ('VC04') (2-6 free parameters)
# However in NLS1 objects, FeII is much stronger, and sometimes more detailed 
# fitting is necessary, use the Kovacevic 2010 template 
# ('K10'; 7 free parameters).

# The options are:
# template   : VC04 (Veron-Cetty 2004) or K10 (Kovacevic 2010)
# amp_const  : constant amplitude (default False)
# disp_const : constant dispersion (default True)
# voff_const : constant velocity offset (default True)
# temp_const : constant temp ('K10' only)

opt_feii_options={
"opt_template"  :{"type":"VC04"}, 
"opt_amp_const" :{"bool":False, "br_opt_feii_val":1.0   , "na_opt_feii_val":1.0},
"opt_disp_const":{"bool":False, "br_opt_feii_val":3000.0, "na_opt_feii_val":500.0},
"opt_voff_const":{"bool":False, "br_opt_feii_val":0.0   , "na_opt_feii_val":0.0},
}
# or
# opt_feii_options={
# "opt_template"  :{"type":"K10"},
# "opt_amp_const" :{"bool":False,"f_feii_val":1.0,"s_feii_val":1.0,"g_feii_val":1.0,"z_feii_val":1.0},
# "opt_disp_const":{"bool":False,"opt_feii_val":1500.0},
# "opt_voff_const":{"bool":False,"opt_feii_val":0.0},
# "opt_temp_const":{"bool":True,"opt_feii_val":10000.0},
# }
################################################################################

############################### UV Iron options ################################
uv_iron_options={
"uv_amp_const"  :{"bool":False, "uv_iron_val":1.0},
"uv_disp_const" :{"bool":False, "uv_iron_val":3000.0},
"uv_voff_const" :{"bool":True,  "uv_iron_val":0.0},
}
################################################################################

########################### Balmer Continuum options ###########################
# For most purposes, only the ratio R, and the overall amplitude are free paramters
# but if you want to go crazy, you can fit everything.
balmer_options = {
"R_const"          :{"bool":True,  "R_val":1.0}, # ratio between balmer continuum and higher-order balmer lines
"balmer_amp_const" :{"bool":False, "balmer_amp_val":1.0}, # amplitude of overall balmer model (continuum + higher-order lines)
"balmer_disp_const":{"bool":True,  "balmer_disp_val":5000.0}, # broadening of higher-order Balmer lines
"balmer_voff_const":{"bool":True,  "balmer_voff_val":0.0}, # velocity offset of higher-order Balmer lines
"Teff_const"       :{"bool":True,  "Teff_val":15000.0}, # effective temperature
"tau_const"        :{"bool":True,  "tau_val":1.0}, # optical depth
}

################################################################################

############################### Plotting options ###############################
plot_options={
"plot_param_hist"    : False,# Plot MCMC histograms and chains for each parameter
"plot_HTML"          : False,# make interactive plotly HTML best-fit plot
}
################################################################################

################################ Output options ################################
output_options={
"write_chain"  : False, # Write MCMC chains for all paramters, fluxes, and
                         # luminosities to a FITS table We set this to false 
                         # because MCMC_chains.FITS file can become very large, 
                         # especially  if you are running multiple objects.  
                         # You only need this if you want to reconstruct full chains 
                         # and histograms. 
"write_options": False,  # output restart file
"verbose"      : False,   # print out all steps of fitting process
}
################################################################################

#### Directory Structure: This is where IFU data will differ from fitting normal 1D spectra

In [4]:
########################## Directory Structure #################################

########################## MaNGA Example Fit ###################################
spec_dir = BADASS_DIR.joinpath("example_spectra","MANGA") # folder with spectra in it
redshifts = [0.0323] # redshifts for each object must be entered manually (for now) since they are not included
                      # in MANGA data
apertures = [[38, 38, 37, 38]] # Square apertures for each cube -- only fit the spectra within the aperture.
                             # number is half the side length of the cube, in pixels
    
# # Get full list of spectrum files - will make sub-directories when decomposing the IFU Cube(s), so it is assumed
# # that the cube FITS files are within spec_dir directly.
spec_loc = natsort.natsorted( glob.glob(str(spec_dir.joinpath('*.fits'))))
spec_loc = spec_loc[:]

################################################################################
print(len(spec_loc))
print(spec_loc)

1
['/Users/remington.sexton/BADASS3/example_spectra/MANGA/manga-9485-12705-LOGCUBE.fits']


#### Run

In [5]:
# Iterate linearly over each MANGA cube to be fit, and fit each 1D spectrum within the cube in parallel
for cube, z, ap in zip(spec_loc, redshifts, apertures):
# for cube, z in zip(spec_loc, redshifts):
    print(cube,z,ap)
    # Unpack the spectra into 1D FITS files
    print(f'Unpacking {cube} into subfolders...')
    # The formats currently supported are 'muse' and 'manga'
    wave,flux,ivar,mask,fwhm_res,binnum,npixels,xpixbin,ypixbin,z,dataid,objname = ifu.prepare_ifu(cube, z, format='manga', 
                                                                                  aperture=ap, 
                                                                                  targetsn = 5.0 ,
                                                                                  snr_threshold = 0.0,
                                                                                  voronoi_binning=False,
#                                                                                   maxbins=500,
                                                                                  use_and_mask = False,
                                                                                  )
    
    # print(flux)
#     # Plot the cube data
    ifu.plot_ifu(cube, wave, flux, ivar, mask, binnum, npixels, xpixbin, ypixbin, z, dataid, ap, objname)
#     ifu.plot_ifu(cube, wave, flux, ivar, mask, binnum, npixels, xpixbin, ypixbin, z, dataid, objname)
    
    cube_subdir = os.path.join(os.path.dirname(cube), cube.split(os.sep)[-1].replace('.fits','')) + os.sep
    # sys.exit()
    if __name__ == "__main__":
        badass.run_BADASS(cube_subdir,
                         nprocesses       = 4,
                         nobj             = (0,2),
                         fit_options      = fit_options,
                         mcmc_options     = mcmc_options,
                         comp_options     = comp_options,
                         test_options     = test_options,
                         # New line options
                         narrow_options   = narrow_options,
                         broad_options    = broad_options,
                         absorp_options   = absorp_options,
                         user_lines       = user_lines,
                         user_constraints = user_constraints,
                         losvd_options    = losvd_options,
                         host_options     = host_options,
                         power_options    = power_options,
                         poly_options     = poly_options,
                         opt_feii_options = opt_feii_options,
                         uv_iron_options  = uv_iron_options,
                         balmer_options   = balmer_options,
                         plot_options     = plot_options,
                         output_options   = output_options,
                         sdss_spec        = False,
                         ifu_spec         = True
                         )

/Users/remington.sexton/BADASS3/example_spectra/MANGA/manga-9485-12705-LOGCUBE.fits 0.0323 [38, 38, 37, 38]
Unpacking /Users/remington.sexton/BADASS3/example_spectra/MANGA/manga-9485-12705-LOGCUBE.fits into subfolders...
Fitting 2 1D spectra
Start process memory: 0.53329101                    

 > Starting fit for spaxel_37_38

 > Starting fit for spaxel_38_38
 - Done fitting spaxel_38_38! 

 - Done fitting spaxel_37_38! 

End process memory: 0.53329510                    


## Now we reconstruct the cube data and make some plots!

In [6]:
import importlib
importlib.reload(ifu)

for cube in spec_loc:
    
    _, _, n = ifu.reconstruct_ifu(cube)
    
    # If you want the fit results to be sa"ved as an animated .mp4 file, change animated=True.
    # This option requires that you have both the python package ffmpeg and the ffmpeg software itself 
    # installed on your system.  See https://www.ffmpeg.org/download.html for details.
    ifu.plot_reconstructed_cube(os.path.join(os.path.dirname(cube), f'MCMC_output_{n}'), animated=False)

113 20


100%|███████████████████████████████████████████| 2/2 [00:00<00:00, 3205.43it/s]
